In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# create directory for data
!mkdir splits

# unzip files from zip folder
!unzip "/content/drive/MyDrive/szum_splits/split1" -d "splits/"

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_normalize = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_normalize = ImageDataGenerator(rescale=1./255)
val_normalize = ImageDataGenerator(rescale=1./255)

In [6]:
main_dir = 'splits/split1/'
train_set = train_normalize.flow_from_directory(directory=main_dir+"train",target_size=(225,225),class_mode='categorical')
val_set = val_normalize.flow_from_directory(directory=main_dir+"val",target_size=(225,225),class_mode="categorical")
test_set = test_normalize.flow_from_directory(directory=main_dir+"test",target_size=(225,225),class_mode="categorical")

Found 1225 images belonging to 3 classes.
Found 153 images belonging to 3 classes.
Found 153 images belonging to 3 classes.


In [7]:
from tensorflow.keras import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D,Conv2D,Dropout

In [8]:
model = Sequential()
model.add(Conv2D(64,(3,3),input_shape=(225,225,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3,activation='softmax'))

In [ ]:
plot_model(model,show_layer_names=True,show_shapes=True)

In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(train_set, epochs=9, batch_size=16, validation_data=test_set, callbacks=[early_stopping])

Epoch 1/9
39/39 [==============================] - 99s 2s/step - loss: 1.6934 - accuracy: 0.4506 - val_loss: 0.7823 - val_accuracy: 0.7320
Epoch 2/9
39/39 [==============================] - 99s 3s/step - loss: 0.7016 - accuracy: 0.6727 - val_loss: 0.6620 - val_accuracy: 0.6928


In [ ]:
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot()
plt.show()

In [ ]:
model.evaluate(val_set)
model.evaluate(test_set)


Predictions: